In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

# Define the folder paths and labels
train_folder = r"C:\Users\ADMIN\Desktop\DATASETS\Project Images 1\train"
test_folder = r"C:\Users\ADMIN\Desktop\DATASETS\Project Images 1\test"
labels = ['Laser Line Detected - No Obstacle Detected', 'Laser Line Detected - Obstacle Detected', 'Laser Line Not Detected']

# Define the batch size and image dimensions
batch_size = 32
img_height = 224
img_width = 224

# Load and preprocess the training images and labels
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# Load and preprocess the test images and labels
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
model = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(labels), activation='softmax')
])

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model for 10 epochs with batch size of 32
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=test_generator)

# Evaluate the model on the test images
loss, accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)

model.save('laser_line_detection_model_new')


Found 956 images belonging to 3 classes.
Found 75 images belonging to 3 classes.
Epoch 1/10
30/30 [==============================] - 177s 6s/step - loss: 1.0842 - accuracy: 0.7636 - val_loss: 0.5188 - val_accuracy: 0.8133
Epoch 2/10
30/30 [==============================] - 203s 7s/step - loss: 0.1770 - accuracy: 0.9477 - val_loss: 0.1765 - val_accuracy: 0.8400
Epoch 3/10
30/30 [==============================] - 201s 7s/step - loss: 0.1267 - accuracy: 0.9550 - val_loss: 0.1208 - val_accuracy: 1.0000
Epoch 4/10
30/30 [==============================] - 203s 7s/step - loss: 0.1125 - accuracy: 0.9613 - val_loss: 0.1376 - val_accuracy: 1.0000
Epoch 5/10
30/30 [==============================] - 200s 7s/step - loss: 0.0820 - accuracy: 0.9707 - val_loss: 0.0990 - val_accuracy: 1.0000
Epoch 6/10
30/30 [==============================] - 225s 8s/step - loss: 0.0687 - accuracy: 0.9770 - val_loss: 0.0876 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 221s 7s/step - loss: 

INFO:tensorflow:Assets written to: laser_line_detection_model_new\assets


INFO:tensorflow:Assets written to: laser_line_detection_model_new\assets


In [ ]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np

# Define the folder paths and labels
train_folder = r"C:\Users\ADMIN\Desktop\DATASETS\Project Images 1\train"
labels = ['Laser Line Detected - No Obstacle Detected', 'Laser Line Detected - Obstacle Detected', 'Laser Line Not Detected']

# Load the trained model
model = keras.models.load_model('laser_line_detection_model_new')

# Load and preprocess the user input image
#img_path = input("Enter the image path: ")
img_path = r"C:\Users\ADMIN\Desktop\DATASETS\Project Images 1\Capture - Copy (3).JPG"
img = cv2.imread(img_path)
img = cv2.resize(img, (224, 224))
img = img / 255.0
img = np.expand_dims(img, axis=0)

# Make a prediction on the user input image
predictions = model.predict(img)
predicted_class_index = np.argmax(predictions[0])
predicted_label = labels[predicted_class_index]

# Add the predicted label on top of the user input image
img = cv2.imread(img_path)
img = cv2.putText(img, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Show the image with the predicted label on top
cv2.imshow('Predicted Label', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 388ms/step
